In [149]:
import ROOT
import numpy as np
from hepdata_lib import Submission, Table, Variable, RootFileReader, Uncertainty
from hepdata_lib.helpers import round_value_and_uncertainty, round_value_and_uncertainty_to_decimals
import ROOT as rt
import os

In [150]:
submission = Submission()

## Figure 1 - 3 Postfit Data - MC

In [151]:
chgs = ["+", "-", "z"]
energies = ["5020.0", "13000.0"]

chans = {
    "+": ["we", "wmu"],
    "-": ["we", "wmu"],
    "z": ["ee", "mumu"]
}

def GetChgString(chg):
    if chg == "z":
        return ""
    return "plus" if chg == "+" else "minus"

def GetEnergyString(energy):
    return "5TeV" if energy == "5020.0" else "13TeV"

def GetChannelString(chan):
    if chan == "ee":
        return "ee"
    if chan == "mumu":
        return "mumu"
    if chan == "we":
        return "e"
    if chan == "wmu":
        return "mu"

def GetFigureIndex(chg):
    return chgs.index(chg) + 1

indices = ["a", "b", "c", "d"]

desc_strs = {}
desc_strs['+'] = "Distributions of $m_T$ in the $W^{+}$ signal selection for CHAN final states \
for the pp collisions at $\\sqrt{s}=$ ENERGY after the maximum likelihood fit. \
The EW backgrounds include the contributions from DY, $W\\to\\tau\\nu$, and diboson processes."
desc_strs['-'] = desc_strs['+'].replace("$W^{+}$", "$W^{-}$")
desc_strs['z'] = "Distributions of $m_{ll}$ in the Z signal selection for CHAN final states \
for the pp collisions at $\\sqrt{s}=$ ENERGY after the maximum likelihood fit. \
The EW backgrounds include the contributions from DY, $W\\to l\\nu$, and diboson processes."
    

for chg in chgs:
    idx = 0
    for energy in energies:
        for chan in chans[chg]:
            fig_str = "Figure " + str(GetFigureIndex(chg)) + "-"
            fig_str += indices[idx]
            idx += 1
            
            table = Table(fig_str)
            table.location = fig_str
            
            chg_str = GetChgString(chg)
            energy_str = GetEnergyString(energy)
            chan_str = GetChannelString(chan)
            
            desc = desc_strs[chg].replace("CHAN", chan_str).replace("ENERGY", energy_str)
            print("descriptions: ", desc)
            table.description = desc
            table.keywords["cmenergies"] = [energy]
            if chg != "z":
                table.keywords["observables"] = ["mT"]
            else:
                table.keywords["observables"] = ["mll"]
            
            if chg == "z":
                fname_str = f"{chan_str}__mT0_{energy_str}"
                fname = f"root/histo_zjets_{fname_str}.root"
            else:
                fname_str = f"{chan_str}{chg_str}__mT0_{energy_str}"
                fname = f"root/histo_wjets_{fname_str}.root"
            print(fname)
            
            reader = RootFileReader(fname)
            QCD = reader.read_hist_1d(f"hqcd_{fname_str}")
            round_value_and_uncertainty(QCD)
            round_value_and_uncertainty_to_decimals(QCD, decimals=0)
            TT = reader.read_hist_1d(f"httbar_{fname_str}")
            round_value_and_uncertainty(TT)
            round_value_and_uncertainty_to_decimals(TT, decimals=0)
            EWK = reader.read_hist_1d(f"hewk_{fname_str}")
            round_value_and_uncertainty(EWK)
            round_value_and_uncertainty_to_decimals(EWK, decimals=0)
            SIG = reader.read_hist_1d(f"hsig_{fname_str}")
            round_value_and_uncertainty(SIG)
            round_value_and_uncertainty_to_decimals(SIG, decimals=0)
            Data = reader.read_hist_1d(f"hdata_{fname_str}")
            round_value_and_uncertainty(Data)
            round_value_and_uncertainty_to_decimals(Data, decimals=0)
            ExpTot = reader.read_hist_1d(f"htot_{fname_str}")
            round_value_and_uncertainty(ExpTot)
            round_value_and_uncertainty_to_decimals(ExpTot, decimals=0)
            
            # x and y axis
            if chg != "z":
                xax = Variable("$m_T$", is_independent=True, is_binned=True, units="GeV")
            else:
                xax = Variable("$m_{ll}$", is_independent=True, is_binned=True, units="GeV")
            xax.values = Data["x_edges"]
            yax = Variable("Events", is_independent=False, is_binned=False, units="Events / GeV")
            yax.values = Data["y"]
            
            # read data
            tot = Variable("Number of total expected events after the fit", is_independent=False, is_binned=False, units="Events / GeV")
            tot.values = ExpTot["y"]
            qcd = Variable("Number of QCD multijet events", is_independent=False, is_binned=False, units="Events / GeV")
            qcd.values = QCD["y"]
            tt = Variable("Number of ttbar events", is_independent=False, is_binned=False, units="Events / GeV")
            tt.values = TT["y"]
            ewk = Variable("Number of EWK events", is_independent=False, is_binned=False, units="Events / GeV")
            ewk.values = EWK["y"]
            sig = Variable(f"Number of signal events", is_independent=False, is_binned=False, units="Events / GeV")
            sig.values = SIG["y"]
            data = Variable("Number of data events", is_independent=False, is_binned=False, units="Events / GeV")
            data.values = Data["y"]
            
            # uncertainties 
            unc_tot = Uncertainty("total postfit uncertainty", is_symmetric=True)
            unc_tot.values = ExpTot["dy"]
            unc_data = Uncertainty("Poisson uncertainty", is_symmetric=True)
            unc_data.values = Data["dy"]
            tot.add_uncertainty(unc_tot)
            data.add_uncertainty(unc_data)
            
            # add variables to table
            table.add_variable(xax)
            table.add_variable(yax)
            table.add_variable(tot)
            if chg != "z":
                table.add_variable(qcd)
            table.add_variable(tt)
            table.add_variable(ewk)
            table.add_variable(sig)
            table.add_variable(data)
            
            submission.add_table(table)

descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for e final states for the pp collisions at $\sqrt{s}=$ 5TeV after the maximum likelihood fit. The EW backgrounds include the contributions from DY, $W\to\tau\nu$, and diboson processes.
root/histo_wjets_eplus__mT0_5TeV.root
descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for mu final states for the pp collisions at $\sqrt{s}=$ 5TeV after the maximum likelihood fit. The EW backgrounds include the contributions from DY, $W\to\tau\nu$, and diboson processes.
root/histo_wjets_muplus__mT0_5TeV.root
descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for e final states for the pp collisions at $\sqrt{s}=$ 13TeV after the maximum likelihood fit. The EW backgrounds include the contributions from DY, $W\to\tau\nu$, and diboson processes.
root/histo_wjets_eplus__mT0_13TeV.root
descriptions:  Distributions of $m_T$ in the $W^{+}$ signal selection for mu final states for the pp collisions

## Figure 4 - 9 Measured vs Theory Cross Sections

In [152]:
procnames = ["wplus", "wminus", "z", "wtotal", "wplusoverwminus", "wtotaloverz"]
captions = {}
captions["wplus"] = "$W^{+}\\to\\ell^{+}\\nu$"
captions["wminus"] = "$W^{-}\\to\\ell^{-}\\nu$"
captions["z"] = "$Z\\to\\ell\\ell$"
captions["wtotal"] = "$W\\to\\ell\\nu$"
captions["wplusoverwminus"] = "$W^{+}\\to\\ell^{+}\\nu$ / $W^{-}\\to\\ell^{-}\\nu$"
captions["wtotaloverz"] = "$W\\to\\ell\\nu / Z\\to\\ell\\ell$"

pdfsets = ["NNPDF3.1", "NNPDF4.0", "CT18", "MSHT20"]

sqrtstrs = {}
sqrtstrs["13TeV"] = "at 13TeV"
sqrtstrs["5TeV"] = "at 5.02TeV"
sqrtstrs["sqrtS"] = "between 13TeV and 5.02TeV"

energies = {}
energies["13TeV"] ="13TeV
energies["5TeV"] = "5.02TeV"
energies["sqrtS"] = "13TeV and 5.02TeV"

ifig = 4
for xsectype in ["fid", "inc"]:
    for sqrts in ["13TeV", "5TeV", "sqrtS"]:
        fname = f"txt/{xsectype}/results_{sqrts}_all.txt"
        print("file name: ", fname)
        data = np.loadtxt(fname)
        data = np.genfromtxt(fname, dtype="str")
        
        table_str = "Figure " + str(ifig) 
        table_str += " Fiducial" if xsectype == "fid" else " Total"
        table_str += " cross sections and ratios " + sqrtstrs[sqrts]
        
        table_desc_str = table_str.replace("cross sections and ratios", "cross sections and ratios for $W\\to\\ell\\nu$ and $Z\\to\\ell\\ell$ processes")
        
        table = Table("Figure " + str(ifig))
        table.description = table_desc_str
        table.location = "Figure " + str(ifig)
        ifig += 1
        
        table.keywords["observables"] = ["Cross section"]
        if sqrts == "13TeV":
            table.keywords["cmenergies"] = [13000.0]
        elif sqrts == "5TeV":
            table.keywords["cmenergies"] = [5020.0]
        else:
            table.keywords["cmenergies"] = [5020.0, 13000.0]
        
        for idx in range(6):
            procname = procnames[idx]
            strname = "cross section"
            doratio = True if "over" in procname else False
            strname += "ratio" if doratio else ""
            strname += " for " + captions[procname] + " " + sqrtstrs[sqrts]
            # measured cross section
            strname_tmp = "Measured " + strname
            xsec_measured = Variable(strname_tmp, is_independent=False, is_binned=False, units="pb")
            xsec_measured.values = [data[idx,0]]
            xsec_unc_stat = Uncertainty("statistical uncertainty", is_symmetric=True)
            xsec_unc_stat.values = [data[idx,1]]
            xsec_measured.add_uncertainty(xsec_unc_stat)
            xsec_unc_sys = Uncertainty("systematic uncertainty", is_symmetric=True)
            xsec_unc_sys.values = [data[idx,2]]
            xsec_measured.add_uncertainty(xsec_unc_sys)
            if not doratio:
                xsec_unc_lumi = Uncertainty("luminosity uncertainty", is_symmetric=True)
                xsec_unc_lumi.values = [data[idx,3]]
                xsec_measured.add_uncertainty(xsec_unc_lumi)

            table.add_variable(xsec_measured)

            # theory cross sections
            for ipdf in range(len(pdfsets)):
                icol = 4 + 3 * ipdf
                strname_tmp = "Theoretical " + strname + " with " + pdfsets[ipdf]
                xsec_theory = Variable(strname_tmp, is_independent=False, is_binned=False, units="pb")
                xsec_theory.values = [data[idx,icol]]
                unc_upper = data[idx,icol+1]
                unc_lower = data[idx,icol+2]
                if unc_upper != unc_lower:
                    xsec_unc = Uncertainty("Theoretical uncertainty", is_symmetric=False)
                    xsec_unc.values = [(unc_lower, unc_upper)]
                else:
                    xsec_unc = Uncertainty("Theoretical uncertainty", is_symmetric=True)
                    xsec_unc.values = [unc_lower]
                xsec_theory.add_uncertainty(xsec_unc)
                xsec_theory.add_qualifier("Energy", energies[sqrts])
                table.add_variable(xsec_theory)

        submission.add_table(table)

file name:  txt/fid/results_13TeV_all.txt
file name:  txt/fid/results_5TeV_all.txt
file name:  txt/fid/results_sqrtS_all.txt
file name:  txt/inc/results_13TeV_all.txt
file name:  txt/inc/results_5TeV_all.txt
file name:  txt/inc/results_sqrtS_all.txt


## Figure 10 Cross section as a function of different sqrtS

In [153]:
fname = "root/xsecs.root"
theories = ["Z_ppbar", "W_ppbar", "Z_pp", "Wp_pp", "Wm_pp", "W_pp"]

exps = [
    "g_CMS_13TeV",
    "g_CMS_8TeV",
    "g_CMS_7TeV",
    "g_CMS_5TeV",
    "g_CMS_2760GeV_2",
    "g_CMS_2760GeV",
    "g_CDF_Run2",
    "g_D0_Run1",
    "g_UA2",
    "g_UA1"
]

reader = RootFileReader(fname)
for theo in theories:
    gtheory_name = "g" + theo
    gtheory = reader.read_graph(gtheory_name)
    print(f"theory graph for {gtheory_name} ", gtheory)
    
for exp in exps:
    gexp = reader.read_graph(exp)
    print(f"exp graph for {exp} ", gexp)

theory graph for gZ_ppbar  defaultdict(<class 'list'>, {'x': [0.5, 0.7, 1.0, 1.4, 1.96, 2.0, 2.5, 3.0], 'y': [49.861, 83.16, 128.01, 184.42, 263.41, 269.13, 341.5, 415.8]})
theory graph for gW_ppbar  defaultdict(<class 'list'>, {'x': [0.5, 0.7, 1.0, 1.4, 1.96, 2.0, 2.5, 3.0], 'y': [504.2, 844.0, 1317.6, 1926.8, 2788.0, 2846.0, 3630.0, 4424.0]})
theory graph for gZ_pp  defaultdict(<class 'list'>, {'x': [2.0, 3.0, 4.0, 5.0, 5.02, 6.0, 7.0, 8.0, 10.0, 13.0, 14.0, 20.0], 'y': [207.4, 361.6, 520.5, 681.6, 684.8, 843.6, 1006.0, 1168.0, 1492.0, 1975.0, 2133.0, 3073.0]})
theory graph for gWp_pp  defaultdict(<class 'list'>, {'x': [2.0, 3.0, 4.0, 5.0, 5.02, 6.0, 7.0, 8.0, 9.0, 10.0, 13.0, 14.0, 20.0], 'y': [1544.0, 2515.0, 3478.0, 4432.0, 4451.0, 5375.0, 6308.0, 7230.0, 8141.999999999999, 9050.0, 11753.0, 12608.0, 17573.0]})
theory graph for gWm_pp  defaultdict(<class 'list'>, {'x': [2.0, 3.0, 4.0, 5.0, 5.02, 6.0, 7.0, 8.0, 9.0, 10.0, 13.0, 14.0, 20.0], 'y': [839.2, 1502.0, 2199.0, 2909.0, 2923.

In [154]:
fname = "root/xsecs.root"
reader = RootFileReader(fname)

### Figure 10 (ppbar theory)

In [155]:
# ppbar theory
table = Table("Figure_10_ppbar_theory")
table.location = "Figure_10_ppbar_theory"
table.description = "Summary of theoretical predictions of the total inclusive cross sections for W and Z boson production times leptonic branching fractions at ppbar collisions."
table.keywords["observables"] = ["Theoretical cross section predictions"]
table.keywords["cmenergies"] = sorted([500, 700, 1000, 1400, 1960, 2000, 2500, 3000])

gtheory_Z_ppbar = reader.read_graph("gZ_ppbar")
round_value_and_uncertainty(gtheory_Z_ppbar)
gtheory_W_ppbar = reader.read_graph("gW_ppbar")
round_value_and_uncertainty(gtheory_W_ppbar)

xax_theory_ppbar = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
xax_theory_ppbar.values = gtheory_Z_ppbar["x"]
yax_theory_W_ppbar = Variable("Theoretical cross section times branching fraction for $W\\to\\ell\\nu$ at ppbar collisions", is_independent=False, is_binned=False, units="pb")
yax_theory_W_ppbar.values = gtheory_W_ppbar["y"]
yax_theory_W_ppbar.add_qualifier("Process", "W to lnu")
yax_theory_Z_ppbar = Variable("Theoretical cross section times branching fraction for $Z\\to\\ell\\ell$ at ppbar collisions", is_independent=False, is_binned=False, units="pb")
yax_theory_Z_ppbar.values = gtheory_Z_ppbar["y"]
yax_theory_Z_ppbar.add_qualifier("Process", "Z to ll")
table.add_variable(xax_theory_ppbar)
table.add_variable(yax_theory_W_ppbar)
table.add_variable(yax_theory_Z_ppbar)

submission.add_table(table)

In [156]:
# exp
# UA1 UA2
table = Table("Figure_10_ppbar_exp_UA1UA2")
table.location = "Figure_10_ppbar_exp_UA1UA2"
table.description = "Summary of experimental measurements of the total inclusive cross sections for W and Z boson production times leptonic branching fractions at UA1 and UA2."
table.keywords["observables"] = ["Experimental cross section measurements"]
table.keywords["cmenergies"] = sorted([630.0])

gexp_UA2 = reader.read_graph("g_UA2")
round_value_and_uncertainty(gexp_UA2)
gexp_UA1 = reader.read_graph("g_UA1")
round_value_and_uncertainty(gexp_UA1)

xax_exp = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
xax_exp.values = [gexp_UA2["x"][2]]
yax_exp_W_UA2 = Variable("Cross section times branching fraction for $W\\to\\ell\\nu$ at UA2", is_independent=False, is_binned=False, units="pb")
yax_exp_W_UA2.values = [gexp_UA2["y"][2]]
yax_exp_W_UA2_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_W_UA2_unc.values = [gexp_UA2["dy"][2]]
yax_exp_W_UA2.add_uncertainty(yax_exp_W_UA2_unc)

yax_exp_Z_UA2 = Variable("Cross section times branching fraction for $Z\\to\\ell\\ell$ at UA2", is_independent=False, is_binned=False, units="pb")
yax_exp_Z_UA2.values = [gexp_UA2["y"][3]]
yax_exp_Z_UA2_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Z_UA2_unc.values = [gexp_UA2["dy"][3]]
yax_exp_Z_UA2.add_uncertainty(yax_exp_Z_UA2_unc)

yax_exp_W_UA1 = Variable("Cross section times branching fraction for $W\\to\\ell\\nu$ at UA1", is_independent=False, is_binned=False, units="pb")
yax_exp_W_UA1.values = [gexp_UA1["y"][2]]
yax_exp_W_UA1_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_W_UA1_unc.values = [gexp_UA1["dy"][2]]
yax_exp_W_UA1.add_uncertainty(yax_exp_W_UA1_unc)

yax_exp_Z_UA1 = Variable("Cross section times branching fraction for $Z\\to\\ell\\ell$ at UA1", is_independent=False, is_binned=False, units="pb")
yax_exp_Z_UA1.values = [gexp_UA1["y"][3]]
yax_exp_Z_UA1_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Z_UA1_unc.values = [gexp_UA1["dy"][3]]
yax_exp_Z_UA1.add_uncertainty(yax_exp_Z_UA1_unc)

table.add_variable(xax_exp)
table.add_variable(yax_exp_W_UA2)
table.add_variable(yax_exp_Z_UA2)
table.add_variable(yax_exp_W_UA1)
table.add_variable(yax_exp_Z_UA1)

submission.add_table(table)

In [157]:
table = Table("Figure_10_ppbar_exp_D0")
table.location = "Figure_10_ppbar_exp_D0"
table.description = "Summary of experimental measurements of the total inclusive cross sections for W and Z boson production times leptonic branching fractions at D0."
# D0
gexp_D0 = reader.read_graph("g_D0_Run1")
round_value_and_uncertainty(gexp_D0)
xax_exp_D0 = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
xax_exp_D0.values =[gexp_D0["x"][2]]
yax_exp_W_D0 = Variable("Cross section times branching fraction for $W\\to\\ell\\nu$ at D0", is_independent=False, is_binned=False, units="pb")
yax_exp_W_D0.values = [gexp_D0["y"][2]]
yax_exp_W_D0_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_W_D0_unc.values = [gexp_D0["dy"][2]]
yax_exp_W_D0.add_uncertainty(yax_exp_W_D0_unc)
yax_exp_Z_D0 = Variable("Cross section times branching fraction for $Z\\to\\ell\\ell$ at D0", is_independent=False, is_binned=False, units="pb")
yax_exp_Z_D0.values = [gexp_D0["y"][3]]
yax_exp_Z_D0_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Z_D0_unc.values = [gexp_D0["dy"][3]]
yax_exp_Z_D0.add_uncertainty(yax_exp_Z_D0_unc)

table.add_variable(xax_exp_D0)
table.add_variable(yax_exp_W_D0)
table.add_variable(yax_exp_Z_D0)

submission.add_table(table)

In [158]:
table = Table("Figure_10_ppbar_exp_CDF")
table.location = "Figure_10_ppbar_exp_CDF"
table.description = "Summary of experimental measurements of the total inclusive cross sections for W and Z boson production times leptonic branching fractions at CDF."
# CDF
gexp_CDF = reader.read_graph("g_CDF_Run2")
round_value_and_uncertainty(gexp_CDF)
xax_exp_CDF = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
xax_exp_CDF.values = [gexp_CDF["x"][2]]
yax_exp_W_CDF = Variable("Cross section times branching fraction for $W\\to\\ell\\nu$ at CDF", is_independent=False, is_binned=False, units="pb")
yax_exp_W_CDF.values = [gexp_CDF["y"][2]]
yax_exp_W_CDF_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_W_CDF_unc.values = [gexp_CDF["dy"][2]]
yax_exp_W_CDF.add_uncertainty(yax_exp_W_CDF_unc)
yax_exp_Z_CDF = Variable("Cross section times branching fraction for $Z\\to\\ell\\ell$ at CDF", is_independent=False, is_binned=False, units="pb")
yax_exp_Z_CDF.values = [gexp_CDF["y"][3]]
yax_exp_Z_CDF_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Z_CDF_unc.values = [gexp_CDF["dy"][3]]
yax_exp_Z_CDF.add_uncertainty(yax_exp_Z_CDF_unc)
table.add_variable(xax_exp_CDF)
table.add_variable(yax_exp_W_CDF)
table.add_variable(yax_exp_Z_CDF)
submission.add_table(table)

In [159]:
table = Table("Figure_10_pp_Wxsec_theory")
table.location = "Figure_10_pp_Wxsec_theory"
table.description = "Summary of theoretical predictions of the total inclusive cross sections for W boson production times leptonic branching fractions at pp collisions."
table.keywords["observables"] = ["Theoretical cross section predictions"]
table.keywords["cmenergies"] = list(map(lambda x: x*1000.0, [2.0, 3.0, 4.0, 5.0, 5.02, 6.0, 7.0, 8.0, 10.0, 13.0, 14.0, 20.0]))

## pp theory
gtheory_W_pp = reader.read_graph("gW_pp")
round_value_and_uncertainty(gtheory_W_pp)
gtheory_Wp_pp = reader.read_graph("gWp_pp")
round_value_and_uncertainty(gtheory_Wp_pp)
gtheory_Wm_pp = reader.read_graph("gWm_pp")
round_value_and_uncertainty(gtheory_Wm_pp)

xax_theory_pp = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
print("gtheory_W_pp ", gtheory_W_pp["x"])
xax_theory_pp.values = gtheory_W_pp["x"]

yax_theory_W_pp = Variable("Theoretical cross section times branching fraction for $W\\to\\ell\\nu$ at pp collisions", is_independent=False, is_binned=False, units="pb")
yax_theory_W_pp.values = gtheory_W_pp["y"]
yax_theory_Wp_pp = Variable("Theoretical cross section times branching fraction for $W^{+}\\to\\ell^{+}\\nu$ at pp collisions", is_independent=False, is_binned=False, units="pb")
yax_theory_Wp_pp.values = gtheory_Wp_pp["y"]
yax_theory_Wm_pp = Variable("Theoretical cross section times branching fraction for $W^{-}\\to\\ell^{-}\\nu$ at pp collisions", is_independent=False, is_binned=False, units="pb")
yax_theory_Wm_pp.values = gtheory_Wm_pp["y"]

table.add_variable(xax_theory_pp)
table.add_variable(yax_theory_W_pp)
table.add_variable(yax_theory_Wp_pp)
table.add_variable(yax_theory_Wm_pp)
submission.add_table(table)

table = Table("Figure_10_pp_Zxsec_exp")
table.location = "Figure_10_pp_Zxsec_exp"
table.description = "Summary of experimental measurements of the total inclusive cross sections for Z boson production times leptonic branching fractions at pp collisions."
table.keywords["observables"] = ["Theoretical cross section measurements"]
table.keywords["cmenergies"] = list(map(lambda x: x*1000.0, [2.0, 3.0, 4.0, 5.0, 5.02, 6.0, 7.0, 8.0, 13.0, 14.0, 20.0]))

gtheory_Z_pp = reader.read_graph("gZ_pp")
round_value_and_uncertainty(gtheory_Z_pp)

xax_theory_pp = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
xax_theory_pp.values = gtheory_Z_pp["x"]

yax_theory_Z_pp = Variable("Theoretical cross section times branching fraction for $Z\\to\\ell\\ell$ at pp collisions", is_independent=False, is_binned=False, units="pb")
yax_theory_Z_pp.values = gtheory_Z_pp["y"]

table.add_variable(xax_theory_pp)
table.add_variable(yax_theory_Z_pp)
submission.add_table(table)





gtheory_W_pp  [2.0, 3.0, 4.0, 5.0, 5.02, 6.0, 7.0, 8.0, 9.0, 10.0, 13.0, 14.0, 20.0]


In [160]:
# pp exp (CMS)
W_vals = []
W_uncs = []
Wp_vals = []
Wp_uncs = []
Wm_vals = []
Wm_uncs = []
Z_vals = []
Z_uncs = []

for sqrts in ["2760GeV_2", "5TeV", "7TeV", "8TeV", "13TeV"]:
    gname = "g_CMS_" + sqrts
    gexp = reader.read_graph(gname)
    round_value_and_uncertainty(gexp)
    W_vals.append(gexp["y"][2])
    W_uncs.append(gexp["dy"][2])
    Wp_vals.append(gexp["y"][0])
    Wp_uncs.append(gexp["dy"][0])
    Wm_vals.append(gexp["y"][1])
    Wm_uncs.append(gexp["dy"][1])
    Z_vals.append(gexp["y"][3])
    Z_uncs.append(gexp["dy"][3])

# fix 2760
gname = "g_CMS_2760GeV"
gexp = reader.read_graph(gname)
round_value_and_uncertainty(gexp)
Z_vals[0] = gexp["y"][3]
Z_uncs[0] = gexp["dy"][3]


table = Table("Figure_10_ppbar_exp_CMS")
table.location = "Figure_10_ppbar_exp_CMS"
table.description = "Summary of experimental measurements of the total inclusive cross sections for W and Z boson production times leptonic branching fractions at CMS."

xax_exp_pp = Variable("Center-of-mass energy", is_independent=True, is_binned=False, units="TeV")
xax_exp_pp.values = [2.76, 5.02, 7, 8, 13]

yax_exp_W = Variable("Cross section times branching fraction for $W\\to\\ell\\nu$ at CMS", is_independent=False, is_binned=False, units="pb")
yax_exp_W.values = W_vals
yax_exp_W_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_W_unc.values = W_uncs
yax_exp_W.add_uncertainty(yax_exp_W_unc)

yax_exp_Wp = Variable("Cross section times branching fraction for $W^{+}\\to\\ell^{+}\\nu$ at CMS", is_independent=False, is_binned=False, units="pb")
yax_exp_Wp.values = Wp_vals
yax_exp_Wp_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Wp_unc.values = Wp_uncs
yax_exp_Wp.add_uncertainty(yax_exp_Wp_unc)

yax_exp_Wm = Variable("Cross section times branching fraction for $W^{-}\\to\\ell^{-}\\nu$ at CMS", is_independent=False, is_binned=False, units="pb")
yax_exp_Wm.values = Wm_vals
yax_exp_Wm_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Wm_unc.values = Wm_uncs
yax_exp_Wm.add_uncertainty(yax_exp_Wm_unc)

yax_exp_Z = Variable("Cross section times branching fraction for $Z\\to\\ell\\ell$ at CMS", is_independent=False, is_binned=False, units="pb")
yax_exp_Z.values = Z_vals
yax_exp_Z_unc = Uncertainty("Total uncertainty", is_symmetric=True)
yax_exp_Z_unc.values = Z_uncs
yax_exp_Z.add_uncertainty(yax_exp_Z_unc)

table.add_variable(xax_exp_pp)
table.add_variable(yax_exp_W)
table.add_variable(yax_exp_Wp)
table.add_variable(yax_exp_Wm)
table.add_variable(yax_exp_Z)

submission.add_table(table)

In [161]:
submission.create_files(remove_old=True, outdir="output")

Note that bins with zero content should preferably be omitted completely from the HEPData table.
Note that bins with zero content should preferably be omitted completely from the HEPData table.
Note that bins with zero content should preferably be omitted completely from the HEPData table.
Note that bins with zero content should preferably be omitted completely from the HEPData table.
